In [61]:
import os
import pandas as pd
import numpy as np
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
from google.datalab import storage
from tqdm import tqdm

In [62]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="speech-rec-key.json"

In [63]:
root = "gs://aave-test-bucket"
audio_files = [o.key for o in storage.Bucket('aave-test-bucket').objects() if o.key.startswith('data_processed/wav')]
manifest = pd.read_csv('gs://aave-test-bucket/manifest.csv')
manifest["google_transcription"] = np.nan

In [64]:
def get_single_transcript(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # The language of the supplied audio
    language_code = "en-US"

    config = {
        "language_code": language_code,
    }
    
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    response = operation.result()
    
    transcript = []
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        transcript.append(alternative.transcript.strip())
    
    return " ".join(transcript)

In [65]:
for idx in tqdm(range(len(audio_files))):
    audio_file = audio_files[idx]
    full_file_path = os.path.join(URI_ROOT, audio_file)
    transcript = get_single_transcript(full_file_path)
    manifest.loc[manifest['wav_file'] == audio_file, "google_transcription"] = transcript
    manifest.to_csv("manifest.csv")

100%|██████████| 1120/1120 [2:18:21<00:00,  7.41s/it] 
